In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io

MAIN MISSION: See if it is possible to accuratly predict in which genre is a painting in, if we compare the pictures in certain ways (E.g. colors, contrasts, light)

TODO:
Kuidas ekstraheerida piltidest suvalised ruudud? \
Algoritm: \
    Loeme pildi sisse pikslitega\
    Loome ruudu (kui palju?)\
    Valime ruutude kaupa piltidelt pikslite väärtusi\
    Loeme sisse valitud ruudud (mis on nende kontrastid ruudu siseselt ja ruutude vahel, heledus ja tumedus, valguse kasutamine jne)\

Kuidas kasutame Skimage'it?
    

TODO: Kuidas lugeda sisse erinevat informatsiooni?\
Millist informatsiooni saame erinevatest värviväärtusest? \
Millist informatsiooni annab meile võrdlused ruutude sees ja ruutude vahel?  (Lähikontrast ja üleüldine kontrast?) 

SKIMAGE!\
\
3D numpy array:
  * y coordinate
  * x coordinate
  * RGB (in this sequence)

In [2]:
artists=pd.read_csv("artists.csv",index_col=["id"])
postimp=artists[((artists['genre'].str.contains("Post-Impressionism")==True))]
#We'll drop all the unnecessary columns
postimp.drop(columns=["years","nationality","bio","wikipedia","paintings"])

,name,genre
id,,
8,Vincent van Gogh,Post-Impressionism
28,Henri Matisse,"Impressionism,Post-Impressionism"
33,Henri de Toulouse-Lautrec,Post-Impressionism
35,Camille Pissarro,"Impressionism,Post-Impressionism"
38,Paul Cezanne,Post-Impressionism
41,Georges Seurat,Post-Impressionism
46,Paul Gauguin,"Symbolism,Post-Impressionism"


In [3]:
#reading in all the relevant paintings from the resized 
#we're using the last name
authors=[]
for author in postimp.name:
    authors.append(author.split(" ")[-1])
authors

['Gogh',
 'Matisse',
 'Toulouse-Lautrec',
 'Pissarro',
 'Cezanne',
 'Seurat',
 'Gauguin']

In [8]:
import random
import os

postimp_paintings=pd.DataFrame(columns=["Postimp","Pixeldata","Squaredata"])
for painting in os.listdir("paintings"):
    i=random.randint(0,4)
    if painting.split("_")[-2] in authors:
        new_row = pd.Series(data={"Postimp": 1, "Pixeldata": skimage.io.imread(fname="paintings/" + painting)})
        postimp_paintings=postimp_paintings.append(new_row,ignore_index=True)   
        #Add randomly every fifth painting for better training later
    elif i == 4:
        new_row = pd.Series(data={"Postimp": 0, "Pixeldata": skimage.io.imread(fname="paintings/" + painting)})
        postimp_paintings=postimp_paintings.append(new_row,ignore_index=True)

print(len(postimp_paintings))
print(postimp_paintings.head())

3080
  Postimp                                          Pixeldata Squaredata
0       0  [[[217, 215, 190], [213, 211, 186], [214, 212,...        NaN
1       0  [[223, 230, 236, 232, 231, 246, 253, 244, 234,...        NaN
2       0  [[188, 195, 201, 203, 207, 213, 217, 218, 220,...        NaN
3       0  [[209, 184, 173, 171, 180, 187, 180, 184, 179,...        NaN
4       0  [[245, 246, 248, 250, 252, 252, 251, 249, 243,...        NaN


In [7]:
#Jagame pildi gridiks sellepärast, et saaksime adekvaatselt valida küll suvaliselt ruute sellest gridist, aga et need ei korduks. Vot
#Märgime ära gridi kordinaatide ühised nimetajad, mille järgi saab pildil navigeerida (nt (0,0), mis on esimene ruut vasakul üleval nurgas)
#Kasutame piltide resized versioone, hoida kokku andmete pealt
import random
squaredata =pd.DataFrame(columns=["Squaredata"])
for data in postimp_paintings["Pixeldata"]:
    used_squares=[]
    while len(used_squares)<25:
        y = int(len(data)/20)
        x = int(len(data[0])/20)
        rand_y=random.randint(0,19)
        rand_x=random.randint(0,19)
        if (rand_y,rand_x) not in used_squares:
            used_squares.append((rand_y,rand_x))
            #Now we have the random starting points to the squares which we can use
    squares_with_info=[]
    for start in used_squares:
        for i in range(20):
            one_row_of_pixels=[]
            for j in range(20):
                if not ((start[0]*y+i >= len(data))) | (start[1]*x+j >= len(data[0])):
                    try: 
                        one_row_of_pixels.append(((data[start[0]*y+i][start[1]*x+j][0]),(data[start[0]*y+i][start[1]*x+j][1]),(data[start[0]*y+i][start[1]*x+j][2])))
                    except:
                        continue
            squares_with_info.append(one_row_of_pixels)
    if not (squares_with_info == "nan") | (squares_with_info[0] == []):
        squaredata=squaredata.append({"Squaredata": squares_with_info},ignore_index=True)
postimp_paintings["Squaredata"] = squaredata
postimp_paintings.drop(columns=["Pixeldata"],inplace=True)
postimp_paintings.dropna(inplace=True)
#since we experience some data loss (some paintings are read in in a different manner), we need to drop the remaining rows with NaN values
#squares_with_info is a 2D array, because it's not important for us to know where the square was, but which pixels were in one square


NameError: name 'postimp_paintings' is not defined

In [5]:
data = postimp_paintings.copy()
#pixeldata on juba olemas? aga mingi bug on sees?
data.rename(columns={"Postimp": "Post-Impressionism"})
data = pd.DataFrame(columns=["Post-Impressionism","Pixeldata","Bright_light","Bright_med","Bright_dark", "Color_light","Color_med",
                             "Color_dark", "Colorsq_light","Colorsq_med","Colorsq_dark"])


while i < ((data.shape)[0]):
    bright = brightness(data.iloc[:,1][i])
    if bright == "light":
        data = data.append({'Bright_light' : 1}, {"Bright_med" : 0}, {"Bright_dark", 0}, ignore_index=True)
    elif bright == "medium":
        data = data.append({'Bright_light' : 0}, {"Bright_med" : 1}, {"Bright_dark", 0}, ignore_index=True)
    else:
        data = data.append({'Bright_light' : 0}, {"Bright_med" : 0}, {"Bright_dark", 1}, ignore_index=True)
    
    clr = color_sq(data.iloc[:,1][i])
    if clr == "light":
        data = data.append({'Colorsq_light' : 1}, {"Colorsq_med" : 0}, {"Colorsq_dark", 0}, ignore_index=True)
    elif clr == "medium":
        data = data.append({'Colorsq_light' : 0}, {"Colorsq_med" : 1}, {"Colorsq_dark", 0}, ignore_index=True)
    else:
        data = data.append({'Colorsq_light' : 0}, {"Colorsq_med" : 0}, {"Colorsq_dark", 1}, ignore_index=True)
    
    avg = average(data.iloc[:,1][i])
    if avg == "light":
        data = data.append({'Color_light' : 1}, {"Color_med" : 0}, {"Color_dark", 0}, ignore_index=True)
    elif avg == "medium":
        data = data.append({'Color_light' : 0}, {"Color_med" : 1}, {"Color_dark", 0}, ignore_index=True)
    else:
        data = data.append({'Color_light' : 0}, {"Color_med" : 0}, {"Color_dark", 1}, ignore_index=True)
    i+=1


NameError: name 'postimp_paintings' is not defined

https://www.johndcook.com/blog/2009/08/24/algorithms-convert-color-grayscale/ \
How to turn pictures to grayscale

In [ ]:
#def average(rgb):
 #   return (rgb[0]+rgb[1]+rgb[2])/3


In [14]:
from math import sqrt

#data used in each function is a 2D dataframe which describes the values of the squares selected from the paintings
def brightness(square):
    bright_average = 0
    bright_values = []
    for n in square:
        bright_sum = 0
        for rgb in n:
            n_len = len(n)
            bright_sum += sqrt(0.299 * (rgb[0] * rgb[0]) + 0.587 * (rgb[1] * rgb[1]) + 0.114 * (rgb[2] * rgb[2]))
        bright_values.append(round(bright_sum / n_len))
    for value in bright_values:
        bright_average += value
    bright_average=bright_average/len(bright_values)
    if bright_average >= 10:
        print("light")
        print(bright_average)
        return 'light'
    elif bright_average > 9:
        print("medium")
        print(bright_average)
        return 'medium'
    else:
        print("dark")
        print(bright_average)
        return 'dark'

def brightness0(r,g,b):
    brightness = sqrt(0.299 * (r * r) + 0.587 * (g * g) + 0.114 * (b * b))
    if  brightness > 170:    
        return 'light'
    elif brightness > 60:
        return 'medium'
    else:
        return 'dark'
    
#see funktsioon otsib üles keskmise värvi, aga võtab iga rgb osa ruutu ja siis lõpus leiab ruutjuure
#this function finds the "average color" of the square
def color_sq(square):
    average_colors_sq = []
    for n in square:
        r = 0
        g = 0
        b = 0
        for rgb in n:
            r += rgb[0]*rgb[0]
            g += rgb[1]*rgb[1]
            b += rgb[2]*rgb[2]
        r = round(sqrt(r/len(rgb)))
        g = round(sqrt(g/len(rgb)))
        b = round(sqrt(b/len(rgb)))
        average_colors_sq.append((r,g,b))
    r=0
    g=0
    b=0
    for rgb in average_colors_sq:
        r += rgb[0]
        b += rgb[1]
        g += rgb[2]
    return brightness0((r/len(average_colors_sq),g/len(average_colors_sq),b/len(average_colors_sq)))

#see funktsioon otsib üles keskmise värvi, aga ei võta ruutu
def average(square):
    average_colors_sq = []
    for n in square:
        r = 0
        g = 0
        b = 0
        for rgb in n:
            r += rgb[0]
            g += rgb[1]
            b += rgb[2]
        r = round(sqrt(r/len(rgb)))
        g = round(sqrt(g/len(rgb)))
        b = round(sqrt(b/len(rgb)))
        average_colors_sq.append((r,g,b))
    r=0
    g=0
    b=0
    for rgb in average_colors_sq:
        r += rgb[0]
        b += rgb[1]
        g += rgb[2]
    return brightness0((r/len(average_colors_sq),g/len(average_colors_sq),b/len(average_colors_sq)))

#def rgb2gray(square):
#    grayscaled_square = []
#    for rgb in square:
#       grayscaled_row = []
#       for n in rgb:
#           grayscaled.append((n[0] * 0.2989, n[1] * 0.5870, n[2] * 0.1140))
#       grayscaled_square.append(grayscaled_row)
#   return brightness(grayscaled)



In [4]:
data_copy = data.copy()
train_set = data_copy.sample(frac=0.75, random_state=0)
test_set = data_copy.drop(train_set.index)

NameError: name 'data' is not defined

In [15]:
#Fill out the data dataframe, which we will later use for model training
data
#df['c'] = df.apply(lambda row: row.a + row.b, axis=1)
data["Brightness"] = data.head(100).Squaredata.apply(brightness)
data

light
10.0
light
10.158
light
10.452
light
10.25
light
10.302
light
10.372
medium
9.992
medium
9.464
light
10.184
medium
9.936
light
10.214
medium
9.956
light
10.406
light
10.06
medium
9.394
light
10.252
medium
9.946
light
10.08
light
10.576
light
10.42
light
10.302
light
10.13
medium
9.968
light
10.152
light
10.246
light
10.624
light
10.096
light
10.012
light
10.106
light
10.098
light
10.014
light
10.224
light
10.028
light
10.364
light
10.294
light
10.052
light
10.21
light
10.202
light
10.066
light
10.094
medium
9.956
light
10.018
light
10.218
light
10.058
medium
9.504
light
10.206
light
10.396
medium
9.974
light
10.354
light
10.302
light
10.068
medium
9.984
light
10.264
light
10.11
light
10.168
light
10.162
light
10.102
light
10.356
light
10.022
medium
9.812
light
10.08
medium
9.92
light
10.348
light
10.336
light
10.202
light
10.466
light
10.152
light
10.042
light
10.108
medium
9.644
light
10.296
medium
9.586
light
10.136
light
10.196
light
10.044
medium
9.956
medium
9.94
medium
9.36

,Postimp,Squaredata,Brightness
0,0,"[[(198, 191, 175), (203, 196, 180), (198, 191,...",light
1,0,"[[(81, 59, 45), (66, 47, 32), (60, 45, 26), (6...",light
2,0,"[[(206, 203, 184), (206, 203, 184), (205, 202,...",light
3,0,"[[(219, 196, 164), (207, 184, 152), (217, 194,...",light
4,0,"[[(193, 176, 158), (190, 173, 155), (196, 179,...",light
...,...,...,...
2939,1,"[[(196, 86, 61), (202, 90, 66), (191, 77, 53),...",NaN
2940,1,"[[(208, 202, 206), (208, 202, 204), (207, 202,...",NaN
2941,1,"[[(194, 187, 169), (187, 180, 162), (183, 175,...",NaN
2942,1,"[[(58, 41, 25), (46, 29, 9), (45, 29, 6), (79,...",NaN
